<a href="https://colab.research.google.com/github/manthan-mehta/ImageClassification/blob/master/Features.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np

In [0]:
def rgb2gray(rgb):
  return np.dot(rgb[...,:3],[0.299,0.587,0.144])

In [0]:
def hog_feature(im): #Histogram of Gradient(HOG)
  if im.ndim == 3:
    image = rgb2gray(im)
  else:
    image = np.atleast_2d(im)

  sx,sy = image.shape
  orientations = 9
  cx,cy = (8,8)

  gx = np.zeros(image.shape)
  gy = np.zeros(image.shape)
  gx[:,:-1] = np.diff(image,n=1,axis=0) #gradient in x-direction
  gy[:-1,:] = np.diff(image,n=1,axis=1) #gradient in y-direction

  grad_mag = np.sqrt(gx ** 2 + gy ** 2) # gradient magnitude
  grad_ori = np.arctan2(gy, (gx + 1e-15)) * (180 / np.pi) + 90 # gradient orientation

  n_cellsx = int(np.floor(sx / cx))  # number of cells in x
  n_cellsy = int(np.floor(sy / cy))  # number of cells in y

  #Final feature vector of HOG
  orientation_histogram = np.zeros((n_cellsx, n_cellsy, orientations))
  

